In [1]:
import sqlite3
import pandas as pd

In [2]:
# Read the data from the file
df = pd.read_csv('data.csv')

# Basic Cleaning and EDA

In [3]:
df.head()

,transaction_id,date,category,product,quantity,price
0,1,2024-07-01,Widget,Widget-A,10.0,9.99
1,2,2024-07-01,Gadget,Gadget-X,5.0,19.99
2,3,2024-07-02,Widget,Widget-B,7.0,9.99
3,4,2024-07-02,Doodad,Doodad-1,NaN,4.99
4,5,2024-07-03,Widget,Widget-C,3.0,9.99


In [4]:
# Print the info of the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  50 non-null     int64  
 1   date            50 non-null     object 
 2   category        50 non-null     object 
 3   product         49 non-null     object 
 4   quantity        45 non-null     float64
 5   price           50 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 2.5+ KB


In [5]:
# Convert 'Price' to numeric, setting errors='coerce' turns invalid values into NaN
df['price'] = pd.to_numeric(df['price'], errors='coerce')

In [6]:
# Drop columns with NaN in price and quantity
df = df.dropna(subset=['quantity', 'price'], how = 'all')

In [7]:
# Fill missing values in quantity with 0
df['quantity'] = df['quantity'].fillna(0)

In [8]:
# Calculate the median price for each category
medians = df.groupby("category")['price'].median()
# Fill missing values in 'price' with the median of the category
df['price'] = df['price'].fillna(medians)

In [9]:
# Get the total sales by multiplying quantity and price
df['total_sales'] = df['quantity'] * df['price']

In [10]:
# Get week day
df['date'] = pd.to_datetime(df['date'])
df['day_of_week'] = df['date'].dt.day_name()

In [11]:
# Set high volume flag
df['high_volume'] = df['quantity'] > 10

# Transformations

In [12]:
# Get the category mean
df_category_mean = df.groupby(["category","product"])['price'].mean()

In [13]:

# Get the category revenue
df_category_revenue = df.groupby("category")['total_sales'].sum()

In [15]:
# Get the day with highest sales for each category
# Group by category and day_of_week, sum the total_sales (reset index used to keep the columns)
category_day = df.groupby(['category', 'day_of_week'])['total_sales'].sum().reset_index()
# Get the index of the max value for each category
idx = category_day.groupby('category')['total_sales'].transform('max') == category_day['total_sales']
# Filter the dataframe with the idx
df_category_day = category_day[idx]


In [15]:
# Filter outliers (2+ standard deviations from category mean)
# Calculate the mean and standard deviation for each category
stats = df.groupby("category")['quantity'].agg(['mean', 'std']).reset_index()
# Merge stats back into the original DataFrame
df = df.merge(stats, on="category")
# Tag outliers
df['outlier'] = ((df['quantity'] < df['mean'] - 2 * df['std']) |
                    (df['quantity'] > df['mean'] + 2 * df['std']))

In [16]:
# Filter outliers and clean up the DataFrame
df_outliers = df[df['outlier']].drop(columns=['mean', 'std', 'outlier'])
df = df[~df['outlier']].drop(columns=['mean', 'std', 'outlier'])

# Export data

In [17]:
# SQLite database file
db_file = "data.db"

# Connect to the database (creates the file if it doesn't exist)
conn = sqlite3.connect(db_file)

In [18]:
try:
    # Save dataframes as separate tables
    df.to_sql('sales', conn, if_exists='replace', index=False)
    df_outliers.to_sql('outliers', conn, if_exists='replace', index=False)
    df_category_day.to_sql('category_day', conn, if_exists='replace', index=False)
    df_category_mean.to_sql('category_mean', conn, if_exists='replace')
    df_category_revenue.to_sql('category_revenue', conn, if_exists='replace')
    print("DataFrames saved successfully!")
finally:
    # Close the connection
    conn.close()

DataFrames saved successfully!
